In [ ]:
%load_ext autoreload 
%autoreload 2
%matplotlib inline
from __future__ import division
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from utils.pg_tools import PGWrangler
import pandas as pd
pd.options.display.mpl_style = 'default'  # Will raise error because deprecated
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams.update({'font.size': 22})
plt.rcParams['legend.fancybox'] = True
import numpy as np
from matplotlib import gridspec

pgw = PGWrangler()
engine = pgw.engine

In [ ]:
# Load feature table form Karen
df = pd.read_sql_table('master',engine.connect(),'semantic')

In [ ]:
# Select only codes with year > 2012
df = df[df['i_eventnumber'].apply(lambda x: int(x[3:5])>12)]

In [ ]:
sql = """
      select sem.i_eventnumber, 
      comm."COMMENTTEXT" as comment
      from 
      semantic.master as sem,
      luigi_clean_sp.t_incident as inc,
      luigi_clean_sp.t_case as c,
      luigi_clean_sp.t_comment as comm
      where 
      sem.i_eventnumber = inc."INCIDENTN"
      and
      inc."KEY_INCIDENT" = c."KEY_INCIDENT"
      and
      c."KEY_CASE" = comm."KEY_CASE"
      """
comments_df = pd.DataFrame(pgw.execute(sql) ,columns=('i_eventnumber','COMMENT'))

In [ ]:
df = pd.merge(df,comments_df,on='i_eventnumber')

In [ ]:
def refused(x):
    try:
        return ("refused transport" in x) or ("refusal" in x)
    except:
        return False

In [ ]:
df['refused'] = df['COMMENT'].apply(refused)

In [ ]:
df.head(10)

In [ ]:
fp = sum(df[df['refused']==True]['trns_to_hosp']==True)

In [ ]:
tp = sum(df[df['refused']==True]['trns_to_hosp']==False)

In [ ]:
print("FP: %d, TP: %d, FP/TP=%.2f"%(fp,tp,fp/float(tp)))

In [ ]:
sum(df[df['refused']==True]['m_sent']==True)

In [ ]:
sum(df[df['refused']==True]['m_required']==True)

In [ ]:
sum(df['refused'])/sum(df['m_required'])

In [ ]:
sum(df[(df['refused']==True) & (df['m_required']==True)]['trns_to_hosp']==True)

In [ ]:
df[(df['refused']==True) & (df['trns_to_hosp']==True)]['COMMENT'].reset_index().ix[7]['COMMENT'] #6